<a href="https://colab.research.google.com/github/Margo-s-M/Portfolio_Python/blob/master/A_B_Testing(Portfolio_Project_2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Мета проекту** - проаналізувати результати **A/B-тестування** за допомогою статистичних методів в Python ,що демонструє ключові конверсійні метрики.


**Розрахунок статистичної значущості**.

Для аналізу використовуємо набір даних з дата сету "user_acquisition"

Рахуємо статистичну значущість для таких метрик:

**add_payment_info / session**

**add_shipping_info / session**

**begin_checkout / session**

**new_accounts / session**

In [1]:
#Імпорт бібліотек
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

from google.colab import drive

drive.mount("/content/drive")
%cd /content/drive/MyDrive/ab_test
df = pd.read_csv("ab_test.csv")
df.head()

Mounted at /content/drive
/content/drive/MyDrive/ab_test


,date,country,continent,device,channel,test_group,test,event_name,value
0,2020-11-01,Lithuania,Europe,mobile,Organic Search,2,2,new accout,1
1,2020-11-01,El Salvador,Americas,desktop,Social Search,1,2,new accout,1
2,2020-11-01,Slovakia,Europe,mobile,Paid Search,2,2,new accout,1
3,2020-11-01,Lithuania,Europe,desktop,Paid Search,2,2,new accout,1
4,2020-11-02,North Macedonia,Europe,desktop,Direct,1,2,new accout,1


In [2]:
df['event_name'] = df['event_name'].replace('new accout', 'new account')

In [3]:
class ABTestAnalysis:
    """
    Клас для аналізу A/B тестів:
    - Підрахунок конверсій за метриками
    - Перевірка статистичної значущості методом Z-тесту
    - Підтримка гнучких групувань (розрізів)
    """

    def __init__(self, data: pd.DataFrame, group_cols=None, metrics=None, session_col=None):
        """
        Етап 1. Ініціалізація класу.
        """
        self.data = data.copy()
        self.group_cols = group_cols if group_cols else []
        if not isinstance(self.group_cols, list):
            self.group_cols = [self.group_cols]

        self.metrics = metrics if metrics is not None else [
            'add_payment_info', 'add_shipping_info', 'begin_checkout', 'new account'
        ]

        if session_col is None:
            self.data['session_id'] = self.data.index
            self.session_col = 'session_id'
        else:
            self.session_col = session_col

        print("\nІнформація про унікальні значення у ключових колонках:")
        for col in ['device', 'continent', 'channel', 'event_name']:
            if col in self.data.columns:
                print(f" - {col}: {self.data[col].unique()}")

    def _prepare_aggregated_data(self):
        """
        Етап 2. Підготовка агрегованих даних для аналізу конверсій.
        """
        base_group_cols = ['test', 'test_group'] + self.group_cols
        filtered_metrics = self.data[self.data['event_name'].isin(self.metrics)].copy()

        numerator = filtered_metrics.groupby(base_group_cols + ['event_name'])['value'].sum().reset_index(name='num')
        sessions = self.data[self.data['event_name'] == 'session']
        denominator = sessions.groupby(base_group_cols)['value'].sum().reset_index(name='den')

        aggregated = pd.merge(numerator, denominator, on=base_group_cols, how='left')
        return aggregated

    def _calculate_conversion_metrics(self, aggregated_df):
        """
        Етап 3. Обчислення конверсій та змін.
        """
        pivot = aggregated_df.pivot_table(
            index=['test', 'event_name'] + self.group_cols,
            columns='test_group',
            values=['num', 'den'],
            fill_value=0
        )

        pivot.columns = [f"{metric}{group}" for metric, group in pivot.columns]
        pivot = pivot.reset_index()

        pivot['cr1'] = pivot['num1'] / pivot['den1'].replace(0, np.nan)
        pivot['cr2'] = pivot['num2'] / pivot['den2'].replace(0, np.nan)

        pivot['conv_change_abs'] = pivot['cr2'] - pivot['cr1']
        pivot['conv_change_pct'] = (pivot['conv_change_abs'] / pivot['cr1']) * 100

        return pivot

    def _perform_statistical_test(self, row):
        """
        Етап 4. Проведення статистичного тесту (Z-тест для пропорцій).
        """
        counts = np.array([row['num1'], row['num2']])
        nobs = np.array([row['den1'], row['den2']])
        z_stat, p_value = proportions_ztest(counts, nobs)
        return pd.Series({'z_stat': z_stat, 'p_value': p_value})

    def run(self):
        """
        Етап 5. Основний метод запуску аналізу.
        """
        aggregated = self._prepare_aggregated_data()
        metrics_df = self._calculate_conversion_metrics(aggregated)
        stats_df = metrics_df.apply(self._perform_statistical_test, axis=1)

        result = pd.concat([metrics_df, stats_df], axis=1)
        result['significant'] = result['p_value'] < 0.05

        return result

    def save_results(self, filename='ab_test_results.csv'):
        """
        Етап 6. Збереження результатів аналізу у CSV файл.
        """
        results = self.run()
        results.to_csv(filename, index=False, sep=';', decimal='.', encoding='utf-8-sig')
        print(f"Результати успішно збережені у файл: {filename}")

        try:
            from google.colab import files
            files.download(filename)
            print("Файл готовий до скачування.")
        except ImportError:
            print("Файл збережено локально. Завантажте вручну при потребі.")


# ------------------------
#  використання

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
pd.set_option('display.max_colwidth', None)

# Етап 1. Аналіз без деталізації, усі користувачі разом
analyzer_overall = ABTestAnalysis(df)
overall_results = analyzer_overall.run()
print("Загальний результат по всім користувачам:")
print(overall_results)

# Збереження результатів у файл
#analyzer_overall.save_results('ab_results_overall.csv')



Інформація про унікальні значення у ключових колонках:
 - device: ['mobile' 'desktop' 'tablet']
 - continent: ['Europe' 'Americas' 'Asia' 'Africa' 'Oceania' '(not set)']
 - channel: ['Organic Search' 'Social Search' 'Paid Search' 'Direct' 'Undefined']
 - event_name: ['new account' 'session_with_orders' 'session' 'page_view'
 'user_engagement' 'scroll' 'view_item' 'first_visit' 'session_start'
 'view_promotion' 'select_promotion' 'add_payment_info'
 'add_shipping_info' 'begin_checkout' 'view_search_results' 'select_item'
 'add_to_cart' 'click' 'view_item_list']
Загальний результат по всім користувачам:
    test         event_name      den1      den2     num1     num2       cr1       cr2  conv_change_abs  conv_change_pct    z_stat   p_value  significant
0      1   add_payment_info   45362.0   45193.0   1988.0   2229.0  0.043825  0.049322         0.005497        12.542021 -3.924884  0.000087         True
1      1  add_shipping_info   45362.0   45193.0   3034.0   3221.0  0.066884  0.07127

# Звіт A/B тестування

## Загальний огляд

У цьому дослідженні проаналізовано ключові метрики поведінкових подій користувачів у двох тестових групах: контрольній та тестовій. Кількість сесій у групах є приблизно рівною, що забезпечує коректність порівняння.

## Основні результати (загальні по всім користувачам)

- **add_payment_info**: конверсія у тестовій групі зросла на 12.54% (p-value = 0.000087), зміна є статистично значущою.
- **add_shipping_info**: збільшення на 6.56% (p-value = 0.0092), також статистично значуща.
- **begin_checkout**: покращення на 6.66% (p-value = 0.0029), ефект значущий.
- **new account**: спостерігається незначне зниження конверсії (-2.83%) із p-value = 0.1229, тобто статистично незначуще.



- **На десктопах і мобільних** пристроях спостерігається **позитивний вплив** тесту на більшість ключових метрик, включно з збільшенням `add_payment_info` і `begin_checkout`.

## Висновок

Результати A/B тесту демонструють позитивний ефект щодо ключових конверсійних дій, особливо для desktop та mobile аудиторії. При цьому метрика "new account" не показала статистично значущого покращення, що може бути сигналом для додаткового розгляду.

Рекомендації:
- Продовжувати впровадження змін для desktop і mobile користувачів.





# Колонки дата сету `ab_results_overall.csv`


**test**	- Номер експерименту (A/B тесту) до якого належать дані.Просто ідентифікатор тесту (1, 2, 3, 4).

**event_name**	- Назва події (event name).Тип конверсійної події, яку аналізували — наприклад: begin_checkout, add_payment_info .

**den1** -	Denominator (контрольна група)	Кількість користувачів у контрольній групі, які мали можливість здійснити подію	Підрахунок користувачів/сесій у контрольній групі.

**den2**	- Denominator (тестова група)	Кількість користувачів у тестовій групі, які мали можливість здійснити подію	Підрахунок користувачів/сесій у тестовій групі.

**num1** -	Numerator (контрольна група)	Кількість користувачів у контрольній групі, які фактично виконали подію	Підрахунок подій у контрольній групі.

**num2**-	Numerator (тестова група)	Кількість користувачів у тестовій групі, які фактично виконали подію.Підрахунок подій у тестовій групі.

**cr1** -	Conversion Rate (контрольна група)	Конверсійна частка у контрольній групі — тобто який відсоток користувачів зробив подію	cr1 = num1 / den1.

**cr2** -	Conversion Rate (тестова група)	Конверсійна частка у тестовій групі	cr2 = num2 / den2.

**conv_change_abs**	- Absolute Conversion Change	Абсолютна різниця між конверсіями у тестовій та контрольній групах
conv_change_abs = cr2 - cr1.

**conv_change_pct** -	Percentage Conversion Change	Відносна зміна (у %) конверсії тесту порівняно з контролем	conv_change_pct = (conv_change_abs / cr1) * 100.

**z_stat** -	Z-statistic	Статистичне значення z-тесту, що показує наскільки сильно відрізняються пропорції	Розраховується через формулу z-тесту для двох пропорцій.

**p_value**	- P-value	Ймовірність отримати такі результати випадково (за відсутності реального ефекту)	Виводиться з z-статистики.

**significant**	- Statistical Significance (True/False)	Ознака статистичної значущості результату	True, якщо p_value < 0.05, інакше False.

Посилання на дашбор в Tableau Pablic [AB Testing](https://public.tableau.com/shared/TZFRK8QYH?:display_count=n&:origin=viz_share_link)